In [1]:
import pickle
import sys
import os

import numpy as np

# Get the path of the current script (or current working directory)
parent_dir = os.path.dirname(os.getcwd())
# Append the parent directory to sys.path
sys.path.append(parent_dir)

import src
from src.PrePipelineSteps import Pycromanager2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF

from src.PipelineSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection, SimpleCellposeSegmentaion

from src.PostPipelineSteps import BuildPDFReport, SaveSpotDetectionResults, SaveMasksToAnalysis, SendAnalysisToNAS, \
                                    DeleteTempFiles

                                    

from src import Experiment, PipelineSettings, ScopeClass, PipelineDataClass, Pipeline

def save_dict_to_file(dic, location):
    f = open(location,'w')
    f.write(str(dic))
    f.close()

ModuleNotFoundError: No module named 'bigfish'

In [2]:
pipeline_package_name = 'IntronDiffusion_SpotPopulations'

# Initialize Dataclasses

In [3]:
settings = PipelineSettings(return_data_to_NAS=1,
    optimization_segmentation_method='default',
    save_all_imgs=1,
    save_all_filtered_imgs=0,
    threshold_for_spot_detection=None,
    NUMBER_OF_CORES=4,
    save_filtered_images=1,
    remove_z_slices_borders=1,
    remove_out_of_focus_images=1,
    save_pdf_report=1,
    MINIMAL_NUMBER_OF_Z_SLICES_TO_CONSIDER_A_3D_IMAGE=5,
    NUMBER_Z_SLICES_TO_TRIM=0,
    user_select_number_of_images_to_run=100_000,  # TODO: This is bad but I want it to select all and am too lazy
                                                        # to deal with it rn
    minimum_spots_cluster=5,
    show_plots=1,
    CLUSTER_RADIUS=750,  # int(psf_yx*1.5)
    save_all_images=1,
    display_spots_on_multiple_z_planes=0,
    use_log_filter_for_spot_detection=0,
    sharpness_threshold=1.10,
    local_or_NAS=1,  # 0 for local, 1 for NAS
    save_files=1,
    model_nuc_segmentation=None,
    model_cyto_segmentation=None,
    pretrained_model_nuc_segmentation=None,
    pretrained_model_cyto_segmentation=None,
    diameter_nucleus=100,
    diameter_cytosol=200,
    save_masks_as_file=0,
    MINIMAL_NUMBER_OF_PIXELS_IN_MASK=1000,
    MAX_NUM_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD=20,
    MINUMUM_NUMBERs_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD=3,
    connection_config_location=r'/home/formanj/FISH_Processing_JF/FISH_Processing/config.yml', # "C:\Users\Jack\Desktop\config.yml"  '/home/formanj/FISH_Processing_JF/FISH_Processing/config.yml'
    share_name='share')

settings.bigfish_alpha = 0.5

In [4]:
experiment = Experiment(initial_data_location=None,
                        nucChannel=None,
                        cytoChannel=None,
                        FISHChannel=None,
                        voxel_size_z=300)

In [5]:
scope = ScopeClass( voxel_size_yx=130,
                    spot_z=500,
                    spot_yx=360,
                    microscope_saving_format='pycromanager'
                )

In [6]:
data = PipelineDataClass()

In [7]:
setting_dict = settings.__dict__
experiment_dict = experiment.__dict__
scope_dict = scope.__dict__


# Initialize Steps

In [8]:
PrePipelineSteps = [Pycromanager2NativeDataType(), 
                   Make_Output_Dir_JF(), 
                   Make_Analysis_Dir_JF(), 
                   ConsolidateImageShapes(), 
                   AutomaticSpotDetection_JF()]

In [9]:
PipelineSteps = [SimpleCellposeSegmentaion(), BIGFISH_SpotDetection()]

In [10]:
PostPipelineSteps = [BuildPDFReport(), SaveSpotDetectionResults(), SaveMasksToAnalysis(), SendAnalysisToNAS(), DeleteTempFiles()]

# Combine into single Dict

In [11]:
pipeline_dict = {'settings': setting_dict, 
                 'experiment': experiment_dict, 
                 'scope': scope_dict, 
                 'data': data.__dict__,
                 'PrePipelineSteps': [i.__class__.__name__ for i in PrePipelineSteps],
                 'PipelineSteps': [i.__class__.__name__ for i in PipelineSteps],
                 'PostPipelineSteps': [i.__class__.__name__ for i in PostPipelineSteps]}

In [12]:
# Save the instances to pickle files
package_dir = os.path.join(os.path.dirname(os.getcwd()), 'Pipelines')
os.makedirs(package_dir, exist_ok=True)

save_dict_to_file(pipeline_dict, os.path.join(package_dir, f'{pipeline_package_name}.txt'))



In [13]:
os.path.join(package_dir, f'{pipeline_package_name}.txt')

'c:\\Users\\Jack\\Documents\\GitHub\\FISH_Processing\\Pipelines\\Standard_Pipeline.txt'